# ╰(*°▽°*)╯ Import stuff ╰(*°▽°*)╯

In [1]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
# import sklearn
# from sklearn import ensemble
# from sklearn.ensemble import RandomForestClassifier
import warnings
import gc


- import MetabolitesFromInChI.csv

In [2]:
#df = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/Notebooks/MetabolitesFromInChIKey.csv")

dfInchi = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/data/1515_inchi_only_Groomed.csv")
BiGG = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/data/BiGG_mnxm2.csv")

Calc = pd.read_csv("Calc.csv")

#df1515 = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/Notebooks/MetabolitesInChIKeyiML1515.csv")




In [24]:
Preamble = dfInchi.set_index("#ID")
Skullbasher = BiGG.set_index("MNXM")

Hope = pd.merge(Preamble, Skullbasher, left_index = True, right_index = True)




In [25]:
Hope_slim = Hope.drop(columns = ["	Line 1: ","reference", "formula", "InChIKey", "SMILES", "description","charge","mass" ])


In [26]:
Hope_slim = Hope_slim[["BiGG","name","InChI"]]

In [27]:
Hope_slim.to_csv("Calc.csv")

# Calculating features & creating new df with descriptors
- Iterating over InChI's from MetabolitesFromInChI df 
- Storing DescriptorCalculator output to descrs list
- Creating molDes df & appending Descriptors to each column


In [3]:
mol_list = []

for i in Calc.InChI:
    mol = Chem.MolFromInchi(i)
    mol_list.append(mol)


desc_list=[i[0] for i in Descriptors._descList]
descriptor = MoleculeDescriptors.MolecularDescriptorCalculator(desc_list)

descrs =  []

for i in range(len(mol_list)):
    descrs.append(descriptor.CalcDescriptors(mol_list[i]))
molDes = pd.DataFrame(descrs, columns= desc_list)
molDes.head(5)

Traceback (most recent call last):
  File "C:\Users\meide\.conda\envs\rdkit\lib\site-packages\rdkit\ML\Descriptors\MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "C:\Users\meide\.conda\envs\rdkit\lib\site-packages\rdkit\Chem\Descriptors.py", line 208, in FpDensityMorgan1
    return _FingerprintDensity(x, _rdMolDescriptors.GetMorganFingerprint, 1)
  File "C:\Users\meide\.conda\envs\rdkit\lib\site-packages\rdkit\Chem\Descriptors.py", line 204, in _FingerprintDensity
    return float(val) / mol.GetNumHeavyAtoms()
ZeroDivisionError: float division by zero
Traceback (most recent call last):
  File "C:\Users\meide\.conda\envs\rdkit\lib\site-packages\rdkit\ML\Descriptors\MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "C:\Users\meide\.conda\envs\rdkit\lib\site-packages\rdkit\Chem\Descriptors.py", line 212, in FpDensityMorgan2
    return _FingerprintDensity(x, _rdMolDescriptors.GetMorganFingerprint, 2)
  File "C:\Users\meide\.c

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.000000,0.000000,0.000000,0.000000,0.342643,1.008,0.000,1.007276,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12.382693,-5.621082,12.382693,0.043672,0.069723,663.430,636.214,663.110219,242,0,...,0,0,0,0,0,0,0,0,0,0
2,10.903369,-5.608866,10.903369,0.373579,0.483914,311.187,294.051,311.046597,112,0,...,0,0,0,0,0,0,0,0,0,0
3,9.404861,0.708333,9.404861,0.708333,0.444345,72.107,64.043,72.057515,30,0,...,0,0,0,0,0,0,0,0,0,0
4,10.138871,-0.983021,10.138871,0.015959,0.606252,213.257,196.121,213.124466,86,0,...,0,0,0,0,0,0,0,0,2,0


- Make csv cus reasons

In [5]:
molDes.to_csv("chemical_properties1515.csv", index = False)

In [12]:
# data1 = pd.read_csv("MetabolitesInChIKeyiML1515.csv")
# data2 = pd.read_csv("chemical_properties1515.csv")

# data1 = pd.read_csv("1515_inchi_only_Groomed.csv")
data1 = Calc
data2 = molDes

In [4]:
BiGG = Calc["BiGG"].tolist()
MNXM = Calc["MNXM"].tolist()
# InChI = Calc["InChI"].toList()
# Name = Calc["Name"].toList()

In [6]:
molDes.insert(0, "BiGG", BiGG)
molDes.insert(1, "MNXM", MNXM)

- Concatatenating MetabolitesFromInChIKey & chemical_properties.csv

In [39]:
# result = pd.concat([data1, data2], axis=1).reindex(data2.index)

# result = pd.concat([data1, data2], ignore_index=False, sort=False)


result = pd.merge(data1, data2, on="index")

KeyError: 'index'

In [40]:
result = pd.concat([data1, data2], ignore_index=True, sort=False)

In [9]:
result.to_csv("Chemical_descriptors1515_BiGG.csv", index = False)

In [10]:
df = pd.read_csv("Chemical_descriptors1515.csv")

In [7]:
molDes.to_csv("Calc_results.csv")